In [1]:
import numpy as np
import pandas as pd
import datascience
from datascience.predicates import are
import shapely
from shapely.geometry import Polygon
from shapely.geometry import LineString

In [2]:
x = [ -71.20197, -70.96679]
y = [42.291441, 42.420578]
x_cell = 50
y_cell = 50


x_min = -71.20197 #x corresponds to longitude
x_max = -70.96679
y_min = 42.291441 #y corresponds to latitude
y_max = 42.420578

#Create ticks
x_s = np.linspace(x_min, x_max, x_cell + 1)
y_s = np.linspace(y_min, y_max, y_cell + 1)

#Create grid
x_coord, y_coord = np.meshgrid(x_s, y_s)

In [18]:
exp = pd.read_csv("highwaydata/exp_high_sub.csv")
art = pd.read_csv("highwaydata/art_high_sub.csv")
b = art["begin"]
e = art["end"]
beg = exp["begin"]
end = exp["end"]
del exp["Unnamed: 0"]
del exp["Unnamed: 0.1"]
del exp["nan"]
del art["Unnamed: 0"]
del art["Unnamed: 0.1"]
del art["nan"]
del art["end lat"]
del art["end long"]
exp

,Roadway ID,From Measure,TMC,Route Number,Direction,Community(ies),Segment Begin Location,Segment End Location,Length (miles),Lanes,...,PM Average Travel Time Index,PM% Travel Time Index,PM Speed Index,PM Congested Minutes,begin lat,begin long,end lat,end long,begin,end
0,5.0,47688.6400,129+04435,I-90,Eastbound,BOSTON,Arlington St/Tremont St,I-93,0.342808,3.0,...,1.291284,4.583333,0.946517,64.895288,42.372241,-71.097665,42.362453,-71.056141,Arlington St/Tremont St,I-93
1,5.0,48472.9476,129+04918,I-90,Eastbound,BOSTON,I-93,Boston Bypass Rd/Exit 24/Exit 25,0.482877,2.0,...,1.098692,1.724138,1.011303,24.767686,42.362453,-71.056141,42.342350,-71.053041,I-93,Boston Bypass Rd
2,5.0,49572.2367,129+04919,I-90,Eastbound,BOSTON,Boston Bypass Rd/Exit 24/Exit 25,Ted Williams Tunnel,0.240466,2.0,...,1.537295,4.230769,0.795047,76.606515,42.342350,-71.053041,42.356249,-71.027000,Boston Bypass Rd,Ted Williams Tunnel
3,5.0,50002.3641,129+04920,I-90,Eastbound,BOSTON,Ted Williams Tunnel,Ted Williams Tunnel Toll,1.349607,2.0,...,1.426732,2.480000,0.965688,21.820449,42.356249,-71.027000,42.355404,-71.027040,Ted Williams Tunnel,Ted Williams Tunnel Toll
4,5.0,52180.8706,129+04921,I-90,Eastbound,BOSTON,Ted Williams Tunnel Toll,William F McClellan Hwy,1.141239,2.0,...,1.459242,5.181818,0.976534,48.413887,42.355404,-71.027040,42.388294,-71.016369,Ted Williams Tunnel Toll,William F McClellan Hwy
5,6.0,138.1959,129-04920,I-90,Westbound,BOSTON,William F McClellan Hwy,Ted Williams Tunnel Toll,0.823872,2.0,...,1.247436,1.607143,1.442960,79.405890,42.388294,-71.016369,42.355404,-71.027040,William F McClellan Hwy,Ted Williams Tunnel Toll
6,6.0,1450.5922,129-04919,I-90,Westbound,"BOSTON,",Ted Williams Tunnel Toll,Ted Williams Tunnel,1.625028,2.0,...,1.386654,1.621622,0.961547,9.906522,42.355404,-71.027040,42.356249,-71.027000,Ted Williams Tunnel Toll,Ted Williams Tunnel
7,6.0,4099.6887,129-04918,I-90,Westbound,BOSTON,Ted Williams Tunnel,Boston Bypass Rd/Exit 24/Exit 25,0.343075,2.0,...,1.187204,1.409091,1.160523,1.628550,42.356249,-71.027000,42.342350,-71.053041,Ted Williams Tunnel,Boston Bypass Rd
8,6.0,5951.2601,129-04434,I-90,Westbound,BOSTON,I-93,Arlington St/Tremont St,0.243506,3.0,...,1.098773,2.600000,1.051678,19.114578,42.362453,-71.056141,42.372241,-71.097665,I-93,Arlington St/Tremont St
9,7.0,20343.7154,129+04112,I-93,Northbound,BOSTON,Freeport St/Exit 13,Morrissey Blvd/Exit 14,0.095805,4.0,...,1.236711,1.361702,0.940912,2.327213,42.301282,-71.050325,42.303854,-71.047747,Freeport St,Morrissey Blvd


In [10]:
'''Function Description: given the x and y coordinates of a grid, turns each grid element into a polygon. I will move from left to right, bottom to top

Input: x-coords, y-coords is a list of lists of x and y coordinates for the grid

Output: a list of polygons, one for each grid '''

def make_polygon(x_coords, y_coords):
    polygons = []
    for b in range(0, len(y_coords)-1, 1):
        y = tuple([y[0] for y in y_coords[b:b+2]])
        #grabs each x-coordinate pair for each grid element
        for a in range(0, len(x_coords[0])-1, 1):
            grid = []
            x = x_coords[0][a:a+2]
            #initiate list to store points for each grid element
            #for each x-coordinate pair, get all corresponding y-coordinates
            for j in range(0, len(x), 1):
                grid.append((x[j], y[0]))
                grid.append((x[j], y[1]))
                g = [grid[0]] + grid[2:4] + [grid[1]] 
                #append grid points of polygon into the grid list 
            polygon = Polygon(g)
            polygons.append(polygon)
        #turn grid into shapely polygon and append to polygon list
    return(polygons)

polygons = make_polygon(x_coord, y_coord)

In [26]:
#processing for express highways
data_beg = []
data_end = []

begin_lat = exp["begin lat"]
begin_long = exp["begin long"]
end_lat = exp["end lat"]
end_long = exp["end long"]

for x in range(0,len(begin_lat)) :
    data_beg.append([begin_long[x], begin_lat[x]])
    data_end.append([end_long[x], end_lat[x]])

In [39]:
#processing for arterial highways
data_beg_a = []
data_end_a = []

begin_lat_a = art["begin lat"]
begin_long_a = art["begin long"]
end_lat_a = art["end lat"]
end_long_a = art["end long"]

for x in range(0,len(begin_lat_a)) :
    data_beg_a.append([begin_long_a[x], begin_lat_a[x]])
    data_end_a.append([end_long_a[x], end_lat_a[x]])

In [21]:

'''Function Description: given the start and end points of a line and a polygon, says whether the line goes through that grid

Input: start_loc and end_loc are tuples of latitude, longitude. grid is a list of 4 latitude, longitude tuples detailing a rectangular grid

Output: Boolean'''

def make_lines(begin_lst, end_lst):
        return ([LineString([begin_lst[i], end_lst[i]]) for i in np.arange(0, len(begin_lst), 1)])

In [22]:
'''Function Description: given the start and end points of a line and a full grid, tells how many grid elements the line go through and which grids. Grids are assigned left to right, bottom to top, starting with the bottom left grid as grid 0  

Input: line is a linestring shapely object. grid is a list of latitude, longitude tuples detailing a rectangular grid

Output: tuple, first element is the number of grids the line interests, and the second element is a list of the grid numbers that line interests'''

def how_many(line, poly_list):
    grids_thru = 0
    polygon = []
    for poly in poly_list:
        thru = poly.intersects(line)
        inside = poly.contains(line)
        if thru or inside:
            grids_thru += 1
            index = poly_list.index(poly)
            polygon.append(index)
    return (grids_thru, polygon)

In [27]:
lines = make_lines(data_beg, data_end)

grids_thru = []
for i in lines:
    grids_thru.append(how_many(i, polygons))   

num_thru = []
which_thru = []
for i in grids_thru:
    num_thru.append(i[0])
    which_thru.append(i[1])

In [30]:
#in final data frame: data_beg, data_end, thru_grids, the four grid points
d = {"begin loc": data_beg
     , "end loc": data_end
     , "num through": num_thru
     , "grids through": which_thru}

final_exp = pd.DataFrame(data=d)

In [92]:
lines_a = make_lines(data_beg_a, data_end_a)

grids_thru_a = []
for i in lines_a:
    grids_thru_a.append(how_many(i, polygons))    
    
num_thru_a = []
which_thru_a = []
for i in grids_thru_a:
    num_thru_a.append(i[0])
    which_thru_a.append(i[1])

In [150]:
d_a = {"begin loc": data_beg_a
       , "end loc": data_end_a
       , "num through": num_thru_a
       , "grids through": which_thru_a}

final_art = pd.DataFrame(data=d_a)

In [ ]:
final_exp.to_csv("grid_exp.csv")
#final_art.to_csv("grid_art.csv")

In [37]:
#combined_a = art.join(final_art)
combined_e = exp.join(final_exp)
combined_e.to_json("datagrid_exp.json", orient="index")
#combined_a.to_csv("datagrid_art.csv")

In [40]:
#Scratch code

grid = []
x = x_coord[0][0:2]
y = tuple([y[0] for y in y_coord[0:2]])
for j in range(0, len(y)):
    grid.append((x[0], y[j]))
    grid.append((x[1], y[j]))
g= [grid[0]] + grid[2:4] + [grid[1]] 
polygon = Polygon(g)
a = (-71.20197, 41)#(exp["begin lat"][0],  exp["begin long"][0])
b = (-71.20197, 43)#(exp["end lat"][0],  exp["end long"][0])
line = LineString([a, b])
polygon.intersects(line)

#Generalizing a url string to be read, adjusting for varying latitudes and longitudes
#Plan of attack: take the center of the grid and check at that point
#Vertices of the grid:

def points_in_box(min_lon, max_lon, min_lat, max_lat):
    #Take the center point of this grid box
    lon = float((min_lon + max_lon)/2)
    lat = float((min_lat + max_lat)/2)

    #read page
    begin_lat = exp["begin lat"]
    begin_long = exp["begin long"]
    end_lat = exp["end lat"]
    end_long = exp["end long"]
    #This list is going to hold tuples that will be the ordered pairs of all the nearby stops
    data_beg, data_end = [], []
    for x in range(0,len(begin_lat)) :
        data_beg.append(tuple([begin_lat[x], begin_long[x]]))
        data_end.append(tuple([end_lat[x], end_long[x]]))

    #We now have a list of ordered pairs that describe the locations of all the nearby stops
    #Next we have to determine if there are stops contained within the parameters of a given grid box
    #And, if so, count how many
    startsingrid = 0
    endsingrid = 0
    for y in range(0, len(data_beg)):
        if (data_beg[y][0]>= min_lat and data_beg[y][0]<=max_lat) and (data_beg[y][1]>= min_lon and data_beg[y][1]<=max_lon):
            startsingrid = 1 + startsingrid
        if (data_end[y][0]>= min_lat and data_end[y][0]<=max_lat) and (data_end[y][1]>= min_lon and data_end[y][1]<=max_lon):
            endsingrid = 1 + endsingrid

    #Only part that will need to be changed depending on our needs: What is printed/ returned at the end
    #print("Center of Grid: (" + str(lat) + ", " + str(lon) + ")" )
    print("Start Locations in this gridbox: "+ str(startsingrid)+ ", End Locations in this gridbox: "+ str(endsingrid))
#end of method